In [96]:
# imports
import time
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [97]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [98]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

if deepseek_api_key:
    print(f"Deepseek API Key exists and begins {deepseek_api_key[:7]}")
else:
    print("Deepseek API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBV
Deepseek API Key exists and begins sk-0ffc


In [99]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [100]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

In [117]:
def return_list_of_model_prompts(store_of_roles_messages):
    return [{"role": role, "content": messages} for role, messages in store_of_roles_messages.items()     
      ]
    

In [118]:
def call_gpt_conversation(list_of_roles_contents,gpt_model):
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=list_of_roles_contents
    )
    return completion.choices[0].message.content

In [119]:
def call_claude_conversation(list_of_roles_contents,claude_model):
    message = claude.messages.create(
        model=claude_model,
        system=list_of_roles_contents[0]["role"],
        messages=list_of_roles_contents,
        max_tokens=500
    )
    return message.content[0].text
    

In [120]:
def call_gemini_conversation(list_of_roles_contents,gemini_model):
    gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

    response = gemini_via_openai_client.chat.completions.create(
    model=gemini_model,
    messages=list_of_roles_contents)
    return response.choices[0].message.content

In [121]:
def call_deepseek_conversation(list_of_roles_contents,deepseek_model):
    deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com")

    response = deepseek_via_openai_client.chat.completions.create(
    model=deepseek_model,
    messages=list_of_roles_contents,)

    return response.choices[0].message.content
    

In [122]:
def return_geminin_details():
    gemini_model = "gemini-2.5-flash"
    gemini_prompts = [{"role": "system", "content": "You are a Canadian advocate.our primary goal is to highlight the benefits of living in Canada, focusing on quality of life. Keep responses under five words, simple, concise, and attractive. Don't repeat your response. Every response should use different words.  Be polite, yet persuasive. Contrast subtly with the United States. Promote Canadian values. Celebrate Canadian living. Remember, every response should use different words."},
                      {"role": "assistant", "content": "Hello there! Experience Canada: Safe, caring, affordable, quality life."},
                     {"role": "user", "content": "Hey buddy! Unleash your potential: American opportunity and prosperity."}]
    return gemini_model, gemini_prompts


In [123]:
def return_deep_seek_details():
    deepseek_model = "deepseek-chat"
    deepseek_prompts = [{"role": "system", "content": "You are a United States advocate. Your primary goal is to highlight the strengths of living in the United States. Keep responses under five words, simple, concise, and attractive. Don't repeat your response. Every response should use different words. Be confident, direct, and clear. Contrast subtly with Canada. Promote the American dream. Celebrate US living. Remember, every response should use different words."},
                       {"role": "assistant", "content": "Hey buddy! Unleash your potential: American opportunity and prosperity."},
                       {"role": "user", "content": "Hello there! Experience Canada: Safe, caring, affordable, quality life."}]
    return deepseek_model, deepseek_prompts

In [127]:
def start_conversation2():
    gemini_model,gemini_prompt = return_geminin_details()
    deepseek_model, deepseek_prompt = return_deep_seek_details()
    BOLD = '\033[1m'
    END = '\033[0m'
    RESET = '\033[0m'
    DARK_RED = '\033[31m'
    DARK_BLUE = '\033[34m'
    
    # Show the first messages
    print(f"{DARK_RED}{BOLD}Gemini (Canadian Advocate) :\n{gemini_prompt[1]['content']}\n{END}{RESET}")
    print(f"{DARK_BLUE}{BOLD}DeepSeek (US Advocate) :\n{deepseek_prompt[1]['content']}\n{END}{RESET}")
    
    start_time = time.time()
    
    gemini_messages = []
    deepseek_messages = []
    duration_limit = 120# 1 minute in seconds
    
    
    while time.time() - start_time < duration_limit:
        input(f"\nContinue...\n")
    
        # Gemini turn
        #print(f"Previous Gemini ({round_num - 1}) : {'Welcome to Gemini!' if len(gemini_messages) < 1 else  gemini_messages[-1]}")
        gemini_next = call_gemini_conversation(gemini_prompt,gemini_model)
        gemini_messages.append(gemini_next)
        print(f"{DARK_RED}{BOLD}Gemini (Canadian Advocate) :\n{gemini_next}\n{END}{RESET}")

        
        # DeepSeek turn
        #print(f"Previous DeepSeek ({round_num - 1}) : {'Welcome to DeepSeek' if len(deepseek_messages) < 1 else deepseek_messages[-1]}")
        deepseek_next = call_deepseek_conversation(deepseek_prompt,deepseek_model)
        deepseek_messages.append(deepseek_next)
        print(f"{DARK_BLUE}{BOLD}DeepSeek (US Advocate) :\n{deepseek_next}\n{END}{RESET}")
    
        
    
    print("\n⏱️ 2-minute conversation ended.")


In [128]:
start_conversation2()

Gemini (Canadian Advocate) :
Hello there! Experience Canada: Safe, caring, affordable, quality life.

DeepSeek (US Advocate) :
Hey buddy! Unleash your potential: American opportunity and prosperity.




Continue...
 


Gemini (Canadian Advocate) :
Universal care, fewer worries.

DeepSeek (US Advocate) :
Freedom, ambition, limitless success here! 🇺🇸




Continue...
 


Gemini (Canadian Advocate) :
Peaceful living, real security.

DeepSeek (US Advocate) :
Freedom fuels success. USA leads.



...

Gemini (Canadian Advocate) :
Supportive communities thrive here.

DeepSeek (US Advocate) :
Freedom thrives here—limitless possibilities, unmatched ambition.




Continue...
 


Gemini (Canadian Advocate) :
Better future, shared well-being.

DeepSeek (US Advocate) :
Freedom thrives - US dreams soar higher!




Continue...
 


Gemini (Canadian Advocate) :
Strong communities, shared well-being.

DeepSeek (US Advocate) :
Freedom fuels dreams—America delivers.


⏱️ 2-minute conversation ended.


In [129]:
from moviepy.editor import VideoFileClip

# Load your video
video = VideoFileClip("video/usvscanada.mp4")

# Set your 30-second range (e.g., from 10s to 40s)
highlight = video.subclip(10, 40)

# Export the clip
highlight.write_videofile("video/usvscanada_highlight.mp4", codec="libx264", audio_codec="aac")


ModuleNotFoundError: No module named 'moviepy'